# Estatísticas do Jogos da plataforma Alite

  - This file is part of  program Alite.
  - Copyright © 2023  Carlo Oliveira <carlo@nce.ufrj.br>.
  - [Labase](http://labase.selfip.org) - [NCE](http://portal.nce.ufrj.br) - [UFRJ](http://www.ufrj.br).
  - SPDX-License-Identifier: ([GNU General Public License v3.0 or later](https://www.gnu.org/licenses/gpl-3.0-standalone.html)).

## Version and configurations.

 This module retrieves data and plot statistics for the [Alite - Games](https://games.alite.selfip.org).

## [23.07](https://github.com/labase/data_edu/releases)

### Added

- Preliminary stats on Alite games data.
- Classe adicionada para plotar wisconsin (14).



## Recebe dados dos games de alite e converte para dataframe

In [ ]:
import urllib.request, json
import pandas as pd

furl = 'https://games.alite.selfip.org/score/players'
with urllib.request.urlopen(furl) as url:
    data = json.loads(url.read().decode())
    df = pd.DataFrame(data)

df_players = df
print(df)

## Converte lista de dicionários numa lista de nomes dos jogos

In [ ]:
dfg = [[g["game"] for g in t] for t in df.games]
# dfg = pd.DataFrame(dfg)
df['games_l'] = [[g["game"] for g in t] for t in df.games]
df

## Contagem dentra das listas dos jogos

In [ ]:
from datetime import datetime, timedelta
past = datetime.now() - timedelta(days=4)
present = datetime.now()
now = pd.Timestamp.now() #.tz_localize(None).tz_convert(None)
df['eica'] = [sum([1 for g in t  if "game" in g]) for t in df.games_l.tolist()]
df['wcst'] = [sum([1 for g in t  if "wcst" in g]) for t in df.games_l.tolist()]
df["esc"] = df["ano"].apply(lambda x: int(str(x)[3:]))
df["ida"] = df["idade"].apply(lambda x: int(str(x)[4:])+5)
# df["old"] = df["time"].apply(lambda x: pd.to_timedelta(present-datetime.strptime(x[:-13], '%Y-%m-%d %H:%M:%S'), unit='D'))
# df["old"] = df["time"].apply(lambda x: pd.to_timedelta(now-pd.to_datetime(x), unit='D'))
df["old"] = df["time"].apply(lambda x: now - pd.to_datetime(x).tz_localize(None))
df_rec = df.drop(df[(df.old > timedelta(days=4)) | (df["esc"]<6)| (df["esc"]>9)].index, inplace=False)
# df_rec = df_rec.drop(df_rec[df_rec.old > timedelta(days=4)].index, inplace=False)
# df_rec = df_rec.groupby(['name','esc', "ida"], as_index=False).agg({'eica':'sum', 'wcst':'sum', "_id":"first"}).reset_index()
df_rec = df_rec.groupby(['name','esc', "ida", "eica","wcst", "_id"], as_index=False) .agg({"_id":"first"}).reset_index()#.reset_index()
# dfclean = df_rec.drop(columns=["_id", "games", "time", "games_l", "idade", "ano"], inplace=False)
df_rec

## Converte ObjectId para Identificador

In [ ]:
import re
matches = [re.findall(r"'(.+?)'",text)[0] for text in df_rec._id.to_list()]
df_rec["oid"] = matches
dfo = df_rec.drop('_id', axis=1)
 #.drop(df_rec["_id"])
# df_rec.oid[1]
dfo

## Testa a recuperação dos dados de um jogador

In [ ]:
import urllib.request, json
import pandas as pd
# ObjectId('649f0ffab9995c27c863da75')
def dt(g):
    k, l = g["scorer"][0]["time"], g["scorer"][-1]["time"]
    k, l = pd.to_datetime(k).tz_localize(None), pd.to_datetime(l).tz_localize(None)
    return pd.Timedelta(l-k, unit="min").seconds //60

def gm(oid):
    # furl = 'https://games.alite.selfip.org/score/games?oid=6477cf19f626d3cb95e08c92'
    # furl = f'https://games.alite.selfip.org/score/games?oid=649f0ffab9995c27c863da75'
    furl = f'https://games.alite.selfip.org/score/games?oid={oid}'
    with urllib.request.urlopen(furl) as url:
        data = json.loads(url.read().decode())
        # df = pd.DataFrame(data)
        game = max((dt(g)for g in data if g["scorer"] and g["game"] =="game"), default=0)
        wsct = max((dt(g)for g in data if g["scorer"] and g["game"] =="wsct"), default=0)
        return f"{game} {wsct}"
print(gm("649f0ffab9995c27c863da75"))

## Listagem dos tempos de Jogo por jogador

In [ ]:
dfo["ws_ga"] = dfo["oid"].apply(lambda x: gm(x))
dfo[[ 'gt', 'wt']] = dfo['ws_ga'].str.split(' ', expand=True)
dfo['wtmx'] = dfo['wt'].apply(lambda x: int(x))
dfo['gtmx'] = dfo['gt'].apply(lambda x: int(x))
# dfo[['wt', 'gt']] = dfo['ws_ga'].str.extract('(\d+) (\d+)', expand=True)
# def inter(k, l):
#     return int(k), int(l)
# dfo[['wt1', 'gt1']] = dfo.apply(lambda x: inter(x["wt"], x["gt"]), axis=1)
# def inter(ard):
#     return int(ard["wt"]), int(ard["gt"])
# dfo[['wt1', 'gt1']] = dfo.apply(inter, axis=1)

dfo = dfo.groupby(
    ['name','esc', "ida"], as_index=False
).agg({'wtmx':'sum', 'gtmx':'sum', 'eica':'sum', 'wcst':'sum', "esc":"first", "ida":"first"}).reset_index()

dfo
# df.games

# Recuperação e plotagem das jogadas

## Pedido dos dados ao servidor e converte num arcabouço de dados

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
dfx = dfo #.explode('games_l')
# dfm = pd.melt(df, 'games_l')
_ = plt.figure(figsize=(15,8))
chart = sns.barplot(data=dfx, x="name", y="gtmx", hue="ida")
_ = chart.set(title='Contagem dos Jogos', ylabel='Número de Jogos', xlabel="Participantes")
_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')
